In [1]:
#IMPORT LIBRARY
import pandas as pd
import numpy as np
import json

from bokeh.plotting import figure
from bokeh.palettes import Spectral
from bokeh.core.properties import value
from bokeh.io import show, output_file,output_notebook
from bokeh.models import ColumnDataSource, FactorRange, HoverTool, ColumnDataSource, Panel, Tabs,NumeralTickFormatter, Legend 
from bokeh.layouts import gridplot, column, row, WidgetBox, layout
from bokeh.models.widgets import RadioButtonGroup, Dropdown


In [2]:

with open('./dat/grocerylist.json') as data_file:    
    grocerydata = json.load(data_file)
    
groceryTable = []
names = []
prices = []
itemNames = []
categories = []
dates = []
nameSet = set()
categorySet = set()

for name,receiptList in grocerydata.items():
    nameSet.add(name)
    for date,receipt in receiptList.items():
        for item in receipt:
            itemName = item[0]
            price = item[1]/100
            category = item[2]
            if category is not None:
                category = category.lower()
                category = category.capitalize()
            names.append(name)
            prices.append(price)
            itemNames.append(itemName)
            categories.append(category)
            categorySet.add(category)
            dates.append(date)


groceryDict = {'Name' : names, 'Item Name' : itemNames, 'Price': prices, 'Category' :categories, 'Date': dates}

#create dataframe from groceryTable
grocery_df = pd.DataFrame(groceryDict)
grocery_df['Date']= pd.to_datetime(grocery_df['Date']) 
grocery_df['Category'] = grocery_df['Category'].replace('Gen merchandise', 'Miscellaneous').replace('Refrig/frozen', 'Refrig/Frozen')
grocery_df['Month'] = grocery_df['Date'].dt.to_period('M')
grocery_df.fillna(value='None', inplace=True)
#print(nameSet)
#print(categorySet)
#print(grocery_df)

perPersonDict = dict()
for index, row in grocery_df.iterrows():
    rowList = [row['Item Name'], row['Price'], row['Category'], row['Date'], row['Month'].strftime('%b-%Y')]
#     print(rowList)
    if row['Name'] in perPersonDict:
        perPersonDict[row['Name']].append(rowList)
    else:
        perPersonDict[row['Name']] = [rowList]
        
perPersonCategorySpending = dict()
perPersonTopTenDict = dict()
# for person,receiptItems in perPersonDict.items():
#     print(person)
#     print(receiptItems)

#Initialize dicts
for name in nameSet:
    perPersonCategorySpending[name] = dict()
    perPersonTopTenDict[name] = dict();
        
# Frequency and totalSpending
for person,receiptItems in perPersonDict.items():
    for item in receiptItems:
        if item[0] in perPersonTopTenDict[person]:
            perPersonTopTenDict[person][item[0]][0] += 1
            perPersonTopTenDict[person][item[0]][1] += item[1]
        else:
            perPersonTopTenDict[person][item[0]] = [1, item[1]]

cleanCategorySet = set()
# Category
for person,receiptItems in perPersonDict.items():
    for item in receiptItems:
        if item[4] in perPersonCategorySpending[person]:
            if item[2] in perPersonCategorySpending[person][item[4]]:
                perPersonCategorySpending[person][item[4]][item[2]] += item[1]
            else:
                cleanCategorySet.add(item[2])
                perPersonCategorySpending[person][item[4]][item[2]] = item[1]
            
        else:
            perPersonCategorySpending[person][item[4]] = dict()
            perPersonCategorySpending[person][item[4]][item[2]] = item[1]
            

# Agg Category
perPersonCategoryTotalSpending = dict()
for person, perMonth in perPersonCategorySpending.items():
    perPersonCategoryTotalSpending[person] = dict()
    for date, Categories in perMonth.items():
        for cate,spent in Categories.items():
            if cate in perPersonCategoryTotalSpending[person] :
                perPersonCategoryTotalSpending[person][cate] += spent
            else:
                perPersonCategoryTotalSpending[person][cate] = spent
        
            
# print(perPersonTopTenDict)
# print(perPersonCategorySpending)
# print(perPersonCategoryTotalSpending)

In [3]:
#output_file("grocery_receipt_dashboard.html")

nameList = list(nameSet)
cleanCategoryList = list(cleanCategorySet)
reformatDict = { i : [0]*len(nameList) for i in cleanCategoryList }
i = 0

totals = [0]*len(nameList)
for name, cateDict in perPersonCategoryTotalSpending.items():
    for ca in cateDict.items():
        if ca[0] in reformatDict:
            reformatDict[ca[0]][i] = ca[1]
            totals[i] += ca[1]
    i += 1
    
i = 0
percentDict = { i : [0]*len(nameList) for i in cleanCategoryList }
for name, cateDict in perPersonCategoryTotalSpending.items():
    for ca in cateDict.items():
        if ca[0] in percentDict:
            percentDict[ca[0]][i] = ca[1]/totals[i]
    i += 1

# print(reformatDict)
# print(percentDict)
reformatDict['Name'] = nameList
percentDict['Name'] = nameList
reformatedData = []
percentreformatedData = []
for person,cateList in reformatDict.items():
    reformatedData.append(cateList)
    
for person,cateList in percentDict.items():
    percentreformatedData.append(cateList)

# print(reformatedData)
# print(percentreformatedData)
# print(len(reformatedData))
# print(len(percentreformatedData))

{'None': [62.19, 7.2700000000000005, 443.8200000000001, 21.29], 'Produce': [2.71, 3.52, 6.79, 6.0], 'Meat': [0, 0, 14.98, 0], 'Refrig/Frozen': [7.99, 13.49, 0, 0], 'Grocery': [17.46, 23.48, 17.98, 4.970000000000001], 'Miscellaneous': [0, 10.24, 9.5, 0], 'Deli': [0, 0, 0, 8.96], 'Seafood': [4.99, 0, 0, 0], 'Liquor': [17.98, 0, 0, 0], 'Tax': [2.32, 0.83, 0.46, 0]}
{'None': [0.5377896921480457, 0.12357640659527451, 0.8992766397179504, 0.5164968461911693], 'Produce': [0.02343479764787271, 0.05983341832398436, 0.013758028893886894, 0.1455604075691412], 'Meat': [0, 0, 0.030352764776204078, 0], 'Refrig/Frozen': [0.0690937391905915, 0.22930477647458777, 0, 0], 'Grocery': [0.150985818056036, 0.39911609722930474, 0.03643142260855469, 0.12057253760310531], 'Miscellaneous': [0, 0.17406085330613633, 0.01924908313577695, 0], 'Deli': [0, 0, 0, 0.2173702086365842], 'Seafood': [0.04315115876859219, 0, 0, 0], 'Liquor': [0.15548253199584922, 0, 0, 0], 'Tax': [0.0200622621930128, 0.01410844807071222, 0.00

In [4]:
colors = Spectral[10]
p1 = figure(x_range=nameList, plot_width=600, plot_height=500, title = "Total Spending Per Person")
v = p1.vbar_stack(cleanCategoryList, x='Name', width=0.9, source=reformatDict, color=colors)

legend_it = []
for name, vitem in zip(cleanCategoryList, v):
    legend_it.append((name,[vitem]))

legend = Legend(items=legend_it, location=(0, 100))
legend.click_policy="mute"

p1.add_layout(legend, 'left')

tab1 = Panel(child= p1, title="Spending Per Category")

#By percent Graph
p2 = figure(x_range=nameList, plot_width=600, plot_height=500, title = "Percent Allocated Per Category")

v2 = p2.vbar_stack(cleanCategoryList, x='Name', width=0.9, source=percentDict, color=colors)
p2.yaxis.formatter = NumeralTickFormatter(format='0%')

legend_it2 = []
for name, vitem in zip(cleanCategoryList, v2):
    legend_it2.append((name,[vitem]))
    
legend2 = Legend(items=legend_it2, location=(0, 100))
legend2.click_policy="mute"

p2.add_layout(legend2, 'left')

tab2 = Panel(child=p2, title="Category by Percent")

tabs = Tabs(tabs=[ tab1, tab2 ])

show(tabs)

In [49]:
output_file("grocery_receipt_dashboard.html")


colors = Spectral[10]

factors=[]

for name,monthPart in perPersonCategorySpending.items():
    for month, ca in monthPart.items():   
        factors.append((name, month))
# print(factors)

perPersonFactors = dict()
for tup in factors:
    print(tup)
    if tup[0] in perPersonFactors:
        perPersonFactors[tup[0]].append(tup)
    else:
        perPersonFactors[tup[0]] = [tup]
        
# print(factors)
# print(perPersonFactors)
        
perPersonLists = dict()
for name,perMonth in perPersonCategorySpending.items():
    #Initilize lists for each 
    tlen = len(perPersonFactors[name]) 
    perPersonLists[name] = {c : [0] * tlen for c in cleanCategoryList}   
    i = 0
    for date, Categories in perMonth.items():
        for cate,spent in Categories.items():
            perPersonLists[name][cate][i] = spent
        i += 1
        
i = 0   
figureListMonths = []
figureListTop = []
tablist = []
for name, item in perPersonLists.items():
    wow = []
    for tup in perPersonFactors[name]:
        wow.append( tup[0] +' ' + tup[1])
    perPersonData = perPersonLists[name]
    perPersonData['x'] = wow

    sources = ColumnDataSource(data = perPersonData)

    tooltips = []
    for cate in cleanCategoryList:
        at = "@"+cate
        tooltips.append((cate, at))
        
    figureListMonths.append(figure(x_range=wow,  plot_width=600, plot_height=500, title = "By Category Spending Habits", y_axis_label = 'Money Spent($)', tooltips=tooltips))
    v = figureListMonths[i].vbar_stack(cleanCategoryList, x='x', width=0.9, source=sources, color=colors)
   

    legend_it = []

    for catename, vitem in zip(cleanCategoryList, v):
        legend_it.append((catename,[vitem]))

    legend = Legend(items=legend_it, location=(0, 100))
    legend.click_policy="mute"

    figureListMonths[i].add_layout(legend, 'left')
    
    
    tablist.append(Panel(child= figureListMonths[i], title= name))
    

    figureListTop.append(figure(x_range=wow,  plot_width=600, plot_height=500, title = "By Category Spending Habits", y_axis_label = 'Money Spent($)'))
    figureListTop[i].vbar_stack(cleanCategoryList, x='x', width=0.9, source=sources, color=colors)
    
    i += 1



tabs = Tabs(tabs= tablist)

show(tabs)

('Calvin', 'Apr-2019')
('Calvin', 'May-2019')
('Rebecca', 'Feb-2018')
('Rebecca', 'Oct-2018')
('Rebecca', 'Feb-2019')
('Person1', 'Apr-2019')
('Celine', 'Apr-2019')
('Celine', 'May-2019')


In [45]:

#Graphing The Top Ten
# print(perPersonTopTenDict)

newDict = dict()
newDictSorted = dict()
for name, itemList in perPersonTopTenDict.items(): 
    newDict[name] = []
    newDictSorted[name] = []
#     valList = [(Values) for itemName, Values in itemList.items()]
#     print(valList)
    for itemName, infoList in itemList.items():
        print(itemName)
        itemName = itemName.strip()
        if (itemName != "TAX" or itemName != "CRV"):
            
            newDict[name].append((itemName, infoList))
            
    newDictSorted[name] = sorted(newDict[name], key = lambda x: x[1][0], reverse = True)
       
# print(newDictSorted)

topTenDictSorted = dict()

for name,items in newDictSorted.items():
    topTenDictSorted[name] = []
    count = 0
    for item in items:
        if(count < 10):
            topTenDictSorted[name].append(item)
            count+=1

print(topTenDictSorted)


figureListTop = []
tablistTop = []

i=0
for name, items in topTenDictSorted.items():
    itemNameList = []
    costList = []
    for item in items:
        itemNameList.append(item[0])
        costList.append(item[1][1])

    figureListTop.append(figure(x_range=itemNameList,  plot_width=600, plot_height=500, title = "Top Ten Purchased Items", y_axis_label = 'Money Spent($)'))
    figureListTop[i].vbar(x=itemNameList, top = costList, width=0.9)
    
    figureListTop[i].xaxis.major_label_orientation = 20

    tablistTop.append(Panel(child= figureListTop[i], title= name))
    i += 1

tabs = Tabs(tabs= tablistTop)

show(tabs)



PHILZ MANHATTAN NP
SCHOCH RAW MILK NP
BOTTLE DEPOSIT NP
JOE JOES NEAPOLITAN
COLD BREW ORGANIC BLACK
CRV
SIG 100% JCE GRPFT
S SEL COFFEE SUN
SIG COND MILK SWEE
STRAUS MLK RF
DEPOSIT MLK
STG 100% JCE GRPFT
SMOKED SALMON LOX
UT RADISH DAIKON
2 QTY TRUMER PIL
CRY BEER 6 PK TAX
DEEP DISH aye CHIP COOKIE
GUMMY BEARS
AVOCADG EACH HASS
LAYS BARBECUE
LAYS SOUR CREAM
MARIE CALLENDERS
SIG CHESE BL BARRL
CRMY RNCH DP 15 OZ
2 QTY KEWPIE MAY
LUCERNE CHEESE MD
OUI BY YOPLAIT
CLIF BAR CHOC CHIP
LASAGNE ROAST PAN
LRG HASS AVOCADOS
2 QTY MELITTA CO
PEETS BRAZIL
NEW YORK STYLE
AIDELLS SSG PORK
WT MDNGHT SDLS ORANGE
RADISHES
WT ORG GALA APPLES
MR 2 QTY RCYCBLE BA
TAX
E 31865 BAGELS
E 3133113 KS PEPGRINDR
E 1287213 SALMON DIP
E 867221 ORG STACY28Z
E 1068080 PASTURE EGGS
E 22093 KS SHRP CHDR
E 7874 PANCAKE MIX
FE eedl491 OREO
E 1169942 GINGER BEER
E 841942 KS JASM
1040486 21ST VARIETY
6000000000 CA REDEMP VA
512599 KS TOWEL
1053843 RACER 5 24PK
DEMP VA
VARIETY
E 1169666 ORG jRF BEAN
E 44871 SOUMDOGH 2PK
E 1